# IntelHealth Agent SFT 数据生成

使用 GPT-4o-mini 为各个 Agent 生成训练数据

## 流程
1. 配置 API Key
2. 测试生成 100 条，查看成本
3. 确认后批量生成完整数据集

## 1. 安装依赖 & 导入库

In [ ]:
# 安装依赖（如果没有的话）
# !pip install openai tqdm

In [ ]:
import os
import json
import time
from typing import List, Dict
from openai import OpenAI
from tqdm.notebook import tqdm
from IPython.display import display, Markdown

print("✅ 库导入成功")

## 2. 配置 API Key

在下面填入你的 OpenAI API Key

In [ ]:
# ============ 配置区 ============

# OpenAI API Key (填入你的 key)
OPENAI_API_KEY = "sk-proj-xxxxx"  # <-- 替换成你的 API Key

# 模型选择
MODEL = "gpt-4o-mini"  # 便宜且效果不错

# API Base URL (如果用代理的话取消注释)
# BASE_URL = "https://api.openai.com/v1"  # 默认
BASE_URL = None  # 使用默认

# 输出目录
OUTPUT_DIR = "./"

# ================================

# 初始化客户端
client = OpenAI(api_key=OPENAI_API_KEY, base_url=BASE_URL)

# 测试连接
try:
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": "说'连接成功'"}],
        max_tokens=10
    )
    print(f"✅ API 连接成功: {response.choices[0].message.content}")
except Exception as e:
    print(f"❌ API 连接失败: {e}")

## 3. 定义 Prompt 模板

每个 Agent 有不同的 Prompt 模板

In [ ]:
# ============ Prompt 模板 ============

PROMPTS = {
    # Agent 1: 症状结构化 (preprocessAndGuess)
    "preprocess": {
        "name": "症状结构化 (preprocessAndGuess)",
        "system": """你是一个医学数据生成助手。你需要生成用于训练"症状结构化"模型的数据。

任务说明：
- 输入：用户的原始症状描述（包括身体部位、主要症状、其他症状、严重程度、持续时间）
- 输出：结构化的医学描述 (optimized_symptoms) 和 RAG 检索关键词 (rag_keywords)

生成要求：
1. optimized_symptoms 应该是专业、完整的医学症状描述，包含所有关键信息
2. rag_keywords 应该是医学关键词数组，用于后续检索，至少包含身体部位和主要症状
3. 将口语化描述转换为专业术语（如"肚子痛"→"腹痛"）
4. 严重程度用文字描述（1=轻微, 2=较轻, 3=中等, 4=较重, 5=严重）

请生成多样化、真实的医疗场景数据。""",

        "user_template": """请生成 {batch_size} 条"症状结构化"训练数据。

每条数据格式如下：
{{
  "input": {{
    "body_part": "身体部位",
    "symptoms": ["症状1", "症状2"],
    "other_symptoms": "其他症状描述",
    "severity": 1-5,
    "duration": "持续时间代码(lessThan24Hours/1To3Days/4To7Days/1To2Weeks/moreThan2Weeks)"
  }},
  "output": {{
    "optimized_symptoms": "专业的症状描述文本",
    "rag_keywords": ["关键词1", "关键词2", ...]
  }}
}}

要求：
1. 生成多样化的症状组合，覆盖不同身体部位
2. 输入使用口语化描述，输出使用专业医学术语
3. 确保 output 是合理的医学转换结果
4. 直接返回 JSON 数组，不要加任何解释文字或 markdown 标记

请返回一个包含 {batch_size} 条数据的 JSON 数组："""
    },

    # Agent 2: 质量评分 (Critic_preprocess)
    "critic": {
        "name": "质量评分 (Critic_preprocess)",
        "system": """你是一个医学数据生成助手。你需要生成用于训练"诊断质量评分"模型的数据。

任务说明：
- 输入：结构化的症状描述 (optimized_symptoms) 和关键词 (rag_keywords)
- 输出：质量评分 (score: 0-5) 和评价意见 (comment)

评分标准（每项1分，满分5分）：
1. optimized_symptoms 格式规范，是完整通顺的医学描述
2. rag_keywords 包含身体部位关键词
3. rag_keywords 包含主要症状关键词
4. 术语使用专业准确
5. 信息完整，无遗漏重要内容

请生成各种质量等级（好/中/差）的样本。""",

        "user_template": """请生成 {batch_size} 条"质量评分"训练数据。

每条数据格式如下：
{{
  "input": {{
    "optimized_symptoms": "症状描述",
    "rag_keywords": ["关键词1", "关键词2"]
  }},
  "output": {{
    "score": 0-5,
    "comment": "评价意见",
    "isValid": true/false
  }}
}}

要求：
1. 生成不同质量等级的样本（差:0-1分, 中:2-3分, 好:4-5分），比例约 2:3:5
2. comment 要具体说明扣分原因或优点
3. score < 3 时 isValid 为 false
4. 直接返回 JSON 数组，不要加任何解释文字或 markdown 标记

请返回一个包含 {batch_size} 条数据的 JSON 数组："""
    },

    # Agent 3: RAG 评审 (checkRAG)
    "check_rag": {
        "name": "RAG评审 (checkRAG)",
        "system": """你是一个医学数据生成助手。你需要生成用于训练"RAG内容评审"模型的数据。

任务说明：
- 输入：用户症状 (optimized_symptoms) 和 RAG 检索结果 (ragContext)
- 输出：相关性评分 (ragScore: 0-5) 和评价意见 (ragComment)

评分标准（每项1分）：
1. RAG 内容与症状相关
2. 包含有用的医学背景知识
3. 信息准确无明显错误
4. 对诊断有参考价值
5. 内容表述清晰

请生成不同相关度的样本。""",

        "user_template": """请生成 {batch_size} 条"RAG评审"训练数据。

每条数据格式如下：
{{
  "input": {{
    "optimized_symptoms": "患者症状描述",
    "ragContext": "RAG检索返回的医学背景知识（可以是相关的，也可以是不太相关的）"
  }},
  "output": {{
    "ragScore": 0-5,
    "ragComment": "评价意见"
  }}
}}

要求：
1. 生成不同相关度的样本（低:0-2, 中:3, 高:4-5），比例约 2:3:5
2. ragContext 要模拟真实的医学知识库检索结果
3. 包含一些不太相关的负样本
4. 直接返回 JSON 数组，不要加任何解释文字或 markdown 标记

请返回一个包含 {batch_size} 条数据的 JSON 数组："""
    },

    # Agent 4: 诊断生成 (generateDiagnosis) - 核心任务
    "diagnosis": {
        "name": "诊断生成 (generateDiagnosis)",
        "system": """你是一个医学数据生成助手。你需要生成用于训练"诊断生成"模型的数据。

任务说明：
- 输入：患者症状 (optimized_symptoms) 和医学背景 (ragContext)
- 输出：诊断结果（3个可能疾病及概率）、健康建议、简化建议

输出格式：
- results: 3个诊断结果，每个包含 condition(疾病名), probability(概率), description(描述)
- recommendations: 3条详细健康建议（每条>=15字）
- recomm_short: 10条简化建议（每条<=10字）

请确保诊断结果符合医学常识。""",

        "user_template": """请生成 {batch_size} 条"诊断生成"训练数据。

每条数据格式如下：
{{
  "input": {{
    "optimized_symptoms": "患者的详细症状描述",
    "ragContext": "相关的医学背景知识"
  }},
  "output": {{
    "results": [
      {{"condition": "疾病1", "probability": 0.xx, "description": "疾病描述"}},
      {{"condition": "疾病2", "probability": 0.yy, "description": "疾病描述"}},
      {{"condition": "疾病3", "probability": 0.zz, "description": "疾病描述"}}
    ],
    "recommendations": ["详细建议1(>=15字)", "详细建议2", "详细建议3"],
    "recomm_short": ["简化1", "简化2", "简化3", "简化4", "简化5", "简化6", "简化7", "简化8", "简化9", "简化10"]
  }}
}}

要求：
1. 诊断结果要符合医学常识，概率之和约为1
2. 涵盖常见病和一些特殊情况
3. 建议要具体可操作
4. 直接返回 JSON 数组，不要加任何解释文字或 markdown 标记

请返回一个包含 {batch_size} 条数据的 JSON 数组："""
    },

    # Agent 5: 用药建议 (generateDrugRecommendations)
    "drug": {
        "name": "用药建议 (generateDrugRecommendations)",
        "system": """你是一个医学数据生成助手。你需要生成用于训练"用药建议"模型的数据。

任务说明：
- 输入：诊断结果 (condition) 和药物知识库内容 (drugRagContext)
- 输出：用药建议列表

输出格式：
每个药物包含：name(药名), usage(用法用量), notes(注意事项)

请确保用药建议符合医学规范。""",

        "user_template": """请生成 {batch_size} 条"用药建议"训练数据。

每条数据格式如下：
{{
  "input": {{
    "condition": "诊断的疾病名称",
    "drugRagContext": "药物知识库检索结果"
  }},
  "output": {{
    "drugs": [{{
      "condition": "疾病名称",
      "recommended_drugs": [
        {{"name": "药品名", "usage": "用法用量", "notes": "注意事项"}},
        {{"name": "药品名2", "usage": "用法用量", "notes": "注意事项"}}
      ]
    }}]
  }}
}}

要求：
1. 药物名称使用标准名
2. 用法用量要具体
3. 注意事项要包含禁忌症
4. 直接返回 JSON 数组，不要加任何解释文字或 markdown 标记

请返回一个包含 {batch_size} 条数据的 JSON 数组："""
    }
}

print(f"✅ 已加载 {len(PROMPTS)} 个 Agent 的 Prompt 模板")
for key, value in PROMPTS.items():
    print(f"   - {key}: {value['name']}")

## 4. 数据生成函数

In [ ]:
# GPT-4o-mini 价格 (per 1M tokens)
INPUT_PRICE = 0.15 / 1_000_000
OUTPUT_PRICE = 0.60 / 1_000_000

# 统计变量
total_tokens = 0
total_cost = 0.0

def generate_batch(agent: str, batch_size: int = 10) -> List[Dict]:
    """生成一批数据"""
    global total_tokens, total_cost
    
    if agent not in PROMPTS:
        raise ValueError(f"Unknown agent: {agent}")
    
    prompt_config = PROMPTS[agent]
    
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": prompt_config["system"]},
                {"role": "user", "content": prompt_config["user_template"].format(batch_size=batch_size)}
            ],
            temperature=0.8,
            max_tokens=4000,
        )
        
        # 统计 token 使用
        usage = response.usage
        input_tokens = usage.prompt_tokens
        output_tokens = usage.completion_tokens
        total_tokens += input_tokens + output_tokens
        
        # 计算成本
        cost = input_tokens * INPUT_PRICE + output_tokens * OUTPUT_PRICE
        total_cost += cost
        
        # 解析返回的 JSON
        content = response.choices[0].message.content.strip()
        
        # 尝试提取 JSON 数组
        if content.startswith("```"):
            content = content.split("```")[1]
            if content.startswith("json"):
                content = content[4:]
        
        data = json.loads(content)
        return data if isinstance(data, list) else [data]
        
    except json.JSONDecodeError as e:
        print(f"⚠️ JSON 解析失败: {e}")
        print(f"   原始内容: {content[:200]}...")
        return []
    except Exception as e:
        print(f"❌ API 调用失败: {e}")
        return []


def build_human_prompt(input_data: Dict, agent: str) -> str:
    """根据 agent 类型构建用户输入"""
    if agent == "preprocess":
        return f"""请将以下症状信息转换为结构化的医学描述：
身体部位: {input_data.get('body_part', '')}
主要症状: {', '.join(input_data.get('symptoms', []))}
其他症状: {input_data.get('other_symptoms', '')}
严重程度: {input_data.get('severity', 3)}
持续时间: {input_data.get('duration', '')}

请输出 JSON 格式：{{"optimized_symptoms": "...", "rag_keywords": [...]}}"""

    elif agent == "critic":
        return f"""请对以下症状结构化结果进行评分：
{json.dumps(input_data, ensure_ascii=False, indent=2)}

请输出 JSON 格式：{{"score": 0-5, "comment": "...", "isValid": true/false}}"""

    elif agent == "check_rag":
        return f"""请评估 RAG 返回内容与症状的相关性：
【症状】{input_data.get('optimized_symptoms', '')}
【RAG内容】{input_data.get('ragContext', '')}

请输出 JSON 格式：{{"ragScore": 0-5, "ragComment": "..."}}"""

    elif agent == "diagnosis":
        return f"""请根据以下信息生成诊断结果：
【症状】{input_data.get('optimized_symptoms', '')}
【医学背景】{input_data.get('ragContext', '')}

请输出诊断 JSON。"""

    elif agent == "drug":
        return f"""请为以下疾病生成用药建议：
【诊断】{input_data.get('condition', '')}
【药物知识】{input_data.get('drugRagContext', '')}

请输出用药建议 JSON。"""

    return json.dumps(input_data, ensure_ascii=False)


def convert_to_training_format(data: List[Dict], agent: str) -> List[Dict]:
    """转换为 SFT 训练格式"""
    training_data = []
    
    for item in data:
        if "input" not in item or "output" not in item:
            continue
        
        human_content = build_human_prompt(item["input"], agent)
        gpt_content = json.dumps(item["output"], ensure_ascii=False)
        
        training_item = {
            "conversations": [
                {"from": "human", "value": human_content},
                {"from": "gpt", "value": gpt_content}
            ]
        }
        training_data.append(training_item)
    
    return training_data


def save_jsonl(data: List[Dict], filepath: str):
    """保存为 JSONL 格式"""
    os.makedirs(os.path.dirname(filepath) if os.path.dirname(filepath) else ".", exist_ok=True)
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    print(f"✅ 已保存到: {filepath}")


print("✅ 函数定义完成")

## 5. 🧪 测试阶段：生成 100 条数据

先生成少量数据，查看：
1. 数据质量
2. 成本估算

In [ ]:
# ============ 测试配置 ============

# 选择要测试的 Agent
TEST_AGENT = "preprocess"  # 可选: preprocess, critic, check_rag, diagnosis, drug

# 测试数量
TEST_NUM_SAMPLES = 100

# 每批生成数量（建议 5-10）
TEST_BATCH_SIZE = 10

# ================================

print(f"📋 测试配置:")
print(f"   Agent: {TEST_AGENT} ({PROMPTS[TEST_AGENT]['name']})")
print(f"   数量: {TEST_NUM_SAMPLES} 条")
print(f"   批次大小: {TEST_BATCH_SIZE}")

In [ ]:
# 重置统计
total_tokens = 0
total_cost = 0.0

# 开始生成
print(f"\n🚀 开始生成 {TEST_AGENT} 测试数据...\n")

all_data = []
num_batches = (TEST_NUM_SAMPLES + TEST_BATCH_SIZE - 1) // TEST_BATCH_SIZE

for i in tqdm(range(num_batches), desc="生成中"):
    current_batch_size = min(TEST_BATCH_SIZE, TEST_NUM_SAMPLES - len(all_data))
    batch_data = generate_batch(TEST_AGENT, current_batch_size)
    all_data.extend(batch_data)
    
    # 避免 rate limit
    if i < num_batches - 1:
        time.sleep(0.5)

print(f"\n✅ 生成完成: {len(all_data)} 条数据")

In [ ]:
# 查看成本统计
print("="*50)
print("📊 成本统计")
print("="*50)
print(f"总 Token 数: {total_tokens:,}")
print(f"测试成本: ${total_cost:.4f}")
print(f"每条成本: ${total_cost/max(len(all_data),1):.6f}")
print()
print("📈 成本预估:")
cost_per_sample = total_cost / max(len(all_data), 1)
print(f"   1,000 条: ${cost_per_sample * 1000:.2f}")
print(f"   5,000 条: ${cost_per_sample * 5000:.2f}")
print(f"  10,000 条: ${cost_per_sample * 10000:.2f}")

In [ ]:
# 查看生成的数据样例
print("📝 数据样例 (前3条):")
print("="*50)

for i, item in enumerate(all_data[:3]):
    print(f"\n--- 样例 {i+1} ---")
    print(json.dumps(item, ensure_ascii=False, indent=2))

In [ ]:
# 保存测试数据
test_output_file = f"{OUTPUT_DIR}/{TEST_AGENT}_test_100.jsonl"

# 转换为训练格式
training_data = convert_to_training_format(all_data, TEST_AGENT)

# 保存
save_jsonl(training_data, test_output_file)

print(f"\n📄 训练格式样例:")
print(json.dumps(training_data[0], ensure_ascii=False, indent=2))

## 6. ✅ 确认后：批量生成完整数据集

如果测试数据质量满意，成本可接受，运行以下代码生成完整数据集

In [ ]:
# ============ 正式生成配置 ============

# 选择要生成的 Agent（可以是列表）
AGENTS_TO_GENERATE = ["preprocess"]  # 可改为 ["preprocess", "critic", "diagnosis"] 等

# 每个 Agent 生成的数量
NUM_SAMPLES = 1000  # 建议先从 1000 开始

# 每批生成数量
BATCH_SIZE = 10

# ====================================

print(f"📋 正式生成配置:")
print(f"   Agents: {AGENTS_TO_GENERATE}")
print(f"   每个 Agent: {NUM_SAMPLES} 条")
print(f"   预估总成本: ${cost_per_sample * NUM_SAMPLES * len(AGENTS_TO_GENERATE):.2f}")
print()
print("⚠️  确认无误后，运行下一个单元格开始生成")

In [ ]:
# 🚀 开始正式生成

for agent in AGENTS_TO_GENERATE:
    print(f"\n{'='*50}")
    print(f"🚀 生成 {agent} ({PROMPTS[agent]['name']}) 数据集")
    print(f"{'='*50}")
    
    # 重置统计
    total_tokens = 0
    total_cost = 0.0
    
    # 生成数据
    all_data = []
    num_batches = (NUM_SAMPLES + BATCH_SIZE - 1) // BATCH_SIZE
    
    for i in tqdm(range(num_batches), desc=f"生成 {agent}"):
        current_batch_size = min(BATCH_SIZE, NUM_SAMPLES - len(all_data))
        batch_data = generate_batch(agent, current_batch_size)
        all_data.extend(batch_data)
        
        if i < num_batches - 1:
            time.sleep(0.5)
    
    # 转换并保存
    training_data = convert_to_training_format(all_data, agent)
    output_file = f"{OUTPUT_DIR}/{agent}_sft_data.jsonl"
    save_jsonl(training_data, output_file)
    
    # 统计
    print(f"\n📊 {agent} 统计:")
    print(f"   生成数量: {len(training_data)} 条")
    print(f"   总 Token: {total_tokens:,}")
    print(f"   成本: ${total_cost:.4f}")

print(f"\n{'='*50}")
print("✅ 所有数据生成完成！")
print(f"{'='*50}")

## 7. 数据验证

In [ ]:
# 验证生成的数据文件
import glob

jsonl_files = glob.glob(f"{OUTPUT_DIR}/*.jsonl")

print("📁 生成的数据文件:")
print("="*50)

for filepath in jsonl_files:
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    print(f"   {os.path.basename(filepath)}: {len(lines)} 条")

In [ ]:
# 随机查看一条数据
import random

# 选择一个文件
if jsonl_files:
    selected_file = jsonl_files[0]
    print(f"📄 随机查看 {os.path.basename(selected_file)} 中的一条数据:\n")
    
    with open(selected_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    random_line = random.choice(lines)
    data = json.loads(random_line)
    print(json.dumps(data, ensure_ascii=False, indent=2))

## 8. 下一步：训练模型

数据生成完成后，可以使用以下命令训练模型：

```bash
cd ../../MyMedicalGPT

python supervised_finetuning.py \
    --model_name_or_path Qwen/Qwen2.5-0.5B-Instruct \
    --train_file_dir ../IntelHealth/data_clean/sft/ \
    --output_dir outputs/loras/preprocess-agent \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --num_train_epochs 3 \
    --learning_rate 2e-4 \
    --use_lora True \
    --lora_rank 64 \
    --lora_alpha 128
```